<a href="https://colab.research.google.com/github/nefro313/hr_finetuning_llama_model/blob/main/Fine_tune_Llama_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pip install datasets huggingface_hub peft bitsandbytes transformers accelerate trl wandb python-dotenv

In [8]:
import os
import wandb
import torch
from dotenv import load_dotenv
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer,SFTConfig

In [9]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"
# The instruction dataset to use
dataset_name = "nefro313/hr_question_llama-2-formatted"

# Fine-tuned model name
new_model = "nefro313/Hr_chatbot_llama-2-7b-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 5

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 2

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 2

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [10]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

data/train/train_dataset.parquet:   0%|          | 0.00/19.7k [00:00<?, ?B/s]

data/test/test_dataset.parquet:   0%|          | 0.00/5.87k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8 [00:00<?, ? examples/s]

In [11]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [12]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [13]:
trainer = SFTTrainer(
    model = model,
    processing_class = tokenizer,
    train_dataset=dataset,
    peft_config=peft_config,
    args =training_arguments,


)

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

In [14]:
# Train model
trainer.train()

Step,Training Loss
25,2.230800


TrainOutput(global_step=30, training_loss=2.189527161916097, metrics={'train_runtime': 295.7038, 'train_samples_per_second': 0.44, 'train_steps_per_second': 0.101, 'total_flos': 1244530857787392.0, 'train_loss': 2.189527161916097, 'epoch': 4.3076923076923075})

In [15]:
trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('nefro313/Hr_chatbot_llama-2-7b-finetune/tokenizer_config.json',
 'nefro313/Hr_chatbot_llama-2-7b-finetune/special_tokens_map.json',
 'nefro313/Hr_chatbot_llama-2-7b-finetune/tokenizer.model',
 'nefro313/Hr_chatbot_llama-2-7b-finetune/added_tokens.json',
 'nefro313/Hr_chatbot_llama-2-7b-finetune/tokenizer.json')

In [16]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "how to introduce ourself in a interveiw"
pipe = pipeline(task="text-generation", model=model ,tokenizer=tokenizer, max_length=500)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] how to introduce ourself in a interveiw [/INST]  Introducing yourself in an interview is a crucial part of the hiring process. Unterscheidung yourself from other candidates by being confident, friendly, and enthusiastic. Here are some tips on how to introduce yourself in an interview:

1. Start with a strong handshake: When you enter the interview room, give a firm handshake with a smile. It shows your confidence and enthusiasm.
2. Be confident: Speak clearly and confidently about your qualifications and experience. Avoid any nervous or stuttering words.
3. Be yourself: Don’t try to be someone you’re not. Be honest and authentic about your qualifications and experience.
4. Show enthusiasm: Show your passion and enthusiasm for the job you’re applying for. Share your interests and motivation for the job.
5. Keep it short and sweet: Keep your introduction short and to the point. Avoid going into too much detail about your personal life.
6. Practice beforehand: Practice your intr

In [17]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

In [18]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [21]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `datasets` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `datasets`


In [22]:
repo_name = 'nefro313/HR-qa-chat-llama2-finetuned-model'
model.push_to_hub(repo_name, check_pr=True)

tokenizer.push_to_hub(repo_name,check_pr=True)

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nefro313/HR-qa-chat-llama2-finetuned-model/commit/edc06db324985fc81847209adc401c8995081af7', commit_message='Upload tokenizer', commit_description='', oid='edc06db324985fc81847209adc401c8995081af7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nefro313/HR-qa-chat-llama2-finetuned-model', endpoint='https://huggingface.co', repo_type='model', repo_id='nefro313/HR-qa-chat-llama2-finetuned-model'), pr_revision=None, pr_num=None)